In [17]:
from PIL import Image
import glob
import numpy as np
import os


def resize_images(img_path):
    images=glob.glob(img_path + "/*.jpg")  
    
    print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
    target_size=(28,28)
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img=new_img.convert('RGB')
        new_img.save(img, "JPEG")
    
    print(len(images), " images resized.")
    return len(images)

def load_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

In [26]:

path_ = '/aiffel/project/exploration_1/data/train/'
num = 0
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + path_ + "scissor"
num += resize_images(image_dir_path)

# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + path_ + "rock"
num += resize_images(image_dir_path)

# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + path_ + "paper"
num += resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + path_
(x_train, y_train)=load_data(image_dir_path, num)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화


480  images to be resized.
480  images resized.
480  images to be resized.
480  images resized.
480  images to be resized.
480  images resized.
학습데이터(x_train)의 이미지 개수는 1440 입니다.


In [30]:

path_ = '/aiffel/project/exploration_1/data/test/'
num = 0
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + path_ + "scissor"
num += resize_images(image_dir_path)

# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + path_ + "rock"
num += resize_images(image_dir_path)

# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + path_ + "paper"
num += resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + path_
(x_test, y_test)=load_data(image_dir_path, num)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화


93  images to be resized.
93  images resized.
93  images to be resized.
93  images resized.
93  images to be resized.
93  images resized.
학습데이터(x_train)의 이미지 개수는 279 입니다.


In [32]:
print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

x_train shape: (1440, 28, 28, 3)
y_train shape: (1440,)


In [196]:
# model을 학습시키는 코드를 직접 작성해 보세요.
# Hint! model.evaluate()을 사용해 봅시다.
# [[YOUR CODE]]
import tensorflow as tf
from tensorflow import keras
import numpy as np

# model을 직접 만들어 보세요.
# Hint! model의 입력/출력부에 특히 유의해 주세요. 가위바위보 데이터셋은 MNIST 데이터셋과 어떤 점이 달라졌나요?
# [[YOUR CODE]]
# 바꿔 볼 수 있는 하이퍼파라미터들
n_channel_1=64
n_channel_2=256
n_channel_3=612
n_dense_1=36
n_dense_2=12
n_train_epoch=5

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense_1, activation='relu'))
model.add(keras.layers.Dense(n_dense_2, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

# model을 학습시키는 코드를 직접 작성해 보세요.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_195 (Conv2D)          (None, 26, 26, 64)        1792      
_________________________________________________________________
max_pooling2d_189 (MaxPoolin (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_196 (Conv2D)          (None, 11, 11, 256)       147712    
_________________________________________________________________
max_pooling2d_190 (MaxPoolin (None, 5, 5, 256)         0         
_________________________________________________________________
conv2d_197 (Conv2D)          (None, 3, 3, 612)         1410660   
_________________________________________________________________
max_pooling2d_191 (MaxPoolin (None, 1, 1, 612)         0         
_________________________________________________________________
flatten_56 (Flatten)         (None, 612)             

In [201]:
cnt_iter = 1
loss_list = []
accuracy_list = []
for i in range(cnt_iter):

    # 모델 훈련
    model.fit(x_train_norm, y_train, epochs=n_train_epoch)

# 모델 시험
# test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
# print("test_loss: {} ".format(test_loss))
# print("test_accuracy: {}".format(test_accuracy))

    test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
    loss_list.append(test_loss)
    accuracy_list.append(test_accuracy)
    print("test_loss: {} ".format(test_loss))
    print("test_accuracy: {}".format(test_accuracy))

print('total_test_loss :', sum(loss_list)/cnt_iter)
print('total_test_accuracy :', sum(accuracy_list)/cnt_iter)

Epoch 1/5
45/45 [==============================] - 0s 5ms/step - loss: 5.6268e-05 - accuracy: 1.0000
Epoch 2/5
45/45 [==============================] - 0s 5ms/step - loss: 4.9788e-05 - accuracy: 1.0000
Epoch 3/5
45/45 [==============================] - 0s 5ms/step - loss: 4.7160e-05 - accuracy: 1.0000
Epoch 4/5
45/45 [==============================] - 0s 5ms/step - loss: 4.3285e-05 - accuracy: 1.0000
Epoch 5/5
45/45 [==============================] - 0s 5ms/step - loss: 3.9851e-05 - accuracy: 1.0000
9/9 - 0s - loss: 0.6604 - accuracy: 0.8459
test_loss: 0.6604313254356384 
test_accuracy: 0.8458781242370605
total_test_loss : 0.6604313254356384
total_test_accuracy : 0.8458781242370605
